In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)
#del df['srno']
#del df['index']


In [4]:
df

,index,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,1,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,2,Axis Direct,Larsen & Toubro,1310.9,1328.10,2019-09-03,1487.00
2,3,Nirmal Bang,Britannia Inds,2695,2704.80,2019-09-03,3220.00
3,4,Sushil Finance,Sahyadri Industries,147,145.50,2019-09-03,220.00
4,5,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...,...
14742,14743,Sharekhan,Hindustan Unilever,2160.15,1922.88,2019-12-18,2185.00
14743,14744,Geojit Financial Services,BPCL,501.65,496.00,2019-12-18,474.00
14744,14745,Prabhudas Lilladher,HDFC,2391.45,2430.80,2019-12-18,2568.00
14745,14746,ICICIdirect.com,Tech Mahindra,814.05,777.44,2019-12-17,900.00


In [5]:
df.drop_duplicates(inplace = True)
df

,index,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,1,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,2,Axis Direct,Larsen & Toubro,1310.9,1328.10,2019-09-03,1487.00
2,3,Nirmal Bang,Britannia Inds,2695,2704.80,2019-09-03,3220.00
3,4,Sushil Finance,Sahyadri Industries,147,145.50,2019-09-03,220.00
4,5,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...,...
14742,14743,Sharekhan,Hindustan Unilever,2160.15,1922.88,2019-12-18,2185.00
14743,14744,Geojit Financial Services,BPCL,501.65,496.00,2019-12-18,474.00
14744,14745,Prabhudas Lilladher,HDFC,2391.45,2430.80,2019-12-18,2568.00
14745,14746,ICICIdirect.com,Tech Mahindra,814.05,777.44,2019-12-17,900.00


In [6]:
#df.to_sql(con=database_connection, name='marketsmojorecos', if_exists='replace')

In [7]:
start_date = '2016-01-01'
end_date = '2016-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [8]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2016 data

In [9]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [10]:
print("No of Brokers in 2016 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 61


## CLEANING

In [11]:
dfsymbols = pd.read_sql("SELECT SYMBOL,recoName FROM symbolsmapped", con=database_connection)
dfsymbols

,SYMBOL,recoName
0,COLPAL,colgatepalmol
1,BRITANNIA,britanniainds
2,TATACHEM,tatachemicals
3,MARUTI,marutisuzuki
4,MANAPPURAM,manappuramfinance
...,...,...
782,None,wpil
783,RPPL,rajshreepolypack
784,SAFARI,safariindustries
785,None,nirlon


In [12]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    appenddf = dfsymbols.loc[dfsymbols['recoName']==sym]
    del appenddf['recoName']
    df3 = df3.append(appenddf,ignore_index=True)
    #sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    #try:
    #    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)
    #    #print(i)
    #except con.Error as err:
    #    if err.errno == connector.errorcode.ER_ACCESS_DENIED_ERROR:
    #        print("Something is wrong with your user name or password")
    #    elif err.errno == errorcode.ER_BAD_DV_ERROR:
    #        print("Database does not exist")
    #    else:
    #        print(err)

In [19]:
df3

,SYMBOL
0,FORCEMOT
1,NH
2,ICICIBANK
3,PARAGMILK
4,KKCL
...,...
3032,IPCALAB
3033,AUROPHARMA
3034,HCLTECH
3035,DHFL


In [20]:
len(df1)

3037

## SYMBOLS ADDED TO TABLE

In [21]:
df4 =pd.concat([df1,df3],axis=1)


In [22]:
df4['predict_date'] = pd.to_datetime(df4['predict_date'], format='%Y-%m-%d')
df4

,reconum,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,234,KRChoksey,Force Motors,1199.9,3497.00,2016-10-03,4820.00,FORCEMOT
1,235,AUM Capital,Narayana Hrudayalay,239,325.55,2016-09-27,400.00,NH
2,236,Religare,ICICI Bank,390.15,217.73,2016-08-04,181.82,ICICIBANK
3,237,Enochventures.com,Parag Milk Foods,137.65,329.60,2016-07-12,425.00,PARAGMILK
4,238,Edelweiss,Kewal Kiran Clothing,1000,1796.00,2016-05-25,2039.00,KKCL
...,...,...,...,...,...,...,...,...
3032,13033,Edelweiss,Ipca Laboratories,957.25,558.00,2016-04-07,440.00,IPCALAB
3033,13034,Nirmal Bang,Aurobindo Pharma,605.95,762.15,2016-04-07,894.00,AUROPHARMA
3034,13035,ICICIdirect.com,HCL Tech.,1108.65,840.00,2016-04-06,1000.00,HCLTECH
3035,13036,Angel Broking,Dewan Hsg Fin. Corp.,47.15,189.00,2016-04-05,270.00,DHFL


In [21]:
uniqueBrokers = df4.broker.unique()
uniqueBrokers

array(['KRChoksey', 'AUM Capital', 'Religare', 'Enochventures.com',
       'Edelweiss', 'Motilal Oswal', 'Kotak Securities', 'Angel Broking',
       'Geojit Financial Services', 'Centrum', 'IIFL', 'ICICIdirect.com',
       'Axis Direct', 'Moneylicious', 'CD Equisearch', 'Nirmal Bang',
       'Arihant Capital Markets', 'Prabhudas Lilladher',
       'Wallfort Financial', 'HDFC Securities', 'BP Wealth',
       'Rudra Shares & Stocks Brokers', 'GEPL Capital', 'Dynamic Levels',
       'Choice Equity Broking', 'Right Horizons',
       'Systematix Shares & Stocks', 'East India Securities Ltd',
       'Sethi Finmart', 'Maximus Securities', 'Ajcon Global',
       'India Nivesh', 'Way2Wealth', 'YES Securities', 'AnandRathi',
       'SPA Securities', 'Khambatta Securities', 'Reliance Securities',
       'Dolat Capital', 'LKP Securities Limited',
       'Cholamandalam Securities', 'JM Financial', 'Ashika Stock Broking',
       'Antique Stock Broking', 'Karvy Stock Broking',
       'Blue Ocean Stra

## PERFORMANCE DEDUCTION ALGORITHM

In [22]:
#del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos


In [17]:
dfprices2016 = pd.read_sql("SELECT * FROM nse2016", con=database_connection)
dfprices2016

,index,Symbol,Date,Closeval
0,0,20MICRONS,2016-01-01,35.20
1,1,3IINFOTECH,2016-01-01,5.15
2,2,3MINDIA,2016-01-01,11321.95
3,3,8KMILES,2016-01-01,2219.55
4,4,A2ZINFRA,2016-01-01,27.60
...,...,...,...,...
390782,390789,NIFTYHIGHBETA50,2016-12-30,1502.89
390783,390790,NIFTYLOWVOL50,2016-12-30,7758.30
390784,390791,NIFTY50VALUE20,2016-12-30,3652.00
390785,390792,NIFTYGROWSECT15,2016-12-30,5196.10


In [18]:
dfprices2017 = pd.read_sql("SELECT * FROM nse2017", con=database_connection)
dfprices2017

,index,Symbol,Date,Closeval
0,0,20MICRONS,2017-01-02,33.85
1,1,3IINFOTECH,2017-01-02,5.60
2,2,3MINDIA,2017-01-02,11056.45
3,3,8KMILES,2017-01-02,674.20
4,4,A2ZINFRA,2017-01-02,42.15
...,...,...,...,...
403386,403393,NIFTYHIGHBETA50,2017-12-29,2396.70
403387,403394,NIFTYLOWVOL50,2017-12-29,10102.21
403388,403395,NIFTY50VALUE20,2017-12-29,4732.00
403389,403396,NIFTYGROWSECT15,2017-12-29,6576.85


In [23]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(type(sym))

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            #dbyear1 = "nse"+str(mydate1.year)
            #dbyear2 = "nse"+str(mydate2.year)
        
            #sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            #dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            dfprices1 = dfprices2016[dfprices2016['Date'] >= mydate1]
            del dfprices1['index']
        
            #sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            #dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            dfprices2 = dfprices2017[dfprices2017['Date'] <= mydate2]
            del dfprices2['index']
            dfprices1 = pd.concat([dfprices1,dfprices2],ignore_index=True)
            dfprices = dfprices1[dfprices1['Symbol'] == sym]
            targetp = float(row['target_price'])
        
            print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                print("HIT!")
                hits = hits + 1
                recbuying = float(row['recomended_buying'])
                maxval = dfprices['Closeval'].max()
                growthperc = (maxval - recbuying) *100 / recbuying
                print('Growth : ',growthperc)
                if np.isnan(growthperc):
                    total = total-1
                else:    
                    if(growthperc < 200):
                        growthSum = growthSum + growthperc
                    else:
                        print("----------------ALERT--------------")
                        high = recbuying * 1.2
                        low = recbuying * 0.8
                        recDate = row['predict_date']
                        print("recDate : ",recDate)
                        dferr = dfprices.loc[dfprices['Date'] == recDate]
                        if dferr['Closeval'].iloc[0] >= low and dferr['Closeval'].iloc[0] <= high :
                            growthSum = growthSum + growthperc
                        else:
                            total = total - 1
                            hits = hits - 1
            else:
                print("DIDN'T HIT!")
            del dfprices
            del dfhits
    if total != 0:
        hitRatio = int(hits*100 / total)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  KRChoksey
<class 'str'>
SYMBOL :  FORCEMOT
DIDN'T HIT!
<class 'str'>
SYMBOL :  SOLARINDS
HIT!
Growth :  80.4476479514416
<class 'str'>
SYMBOL :  BAJFINANCE
HIT!
Growth :  130.15240328253225
<class 'str'>
SYMBOL :  INFIBEAM
HIT!
Growth :  31.394101876675595
<class 'str'>
SYMBOL :  TV18BRDCST
DIDN'T HIT!
<class 'str'>
SYMBOL :  STRTECH
HIT!
Growth :  212.4468085106383
----------------ALERT--------------
recDate :  2016-12-09 00:00:00
<class 'str'>
SYMBOL :  SCHNEIDER
DIDN'T HIT!
<class 'str'>
SYMBOL :  BALKRISIND
HIT!
Growth :  81.16484440706478
<class 'str'>
SYMBOL :  TATAMOTORS
DIDN'T HIT!
<class 'str'>
SYMBOL :  INFIBEAM
HIT!
Growth :  40.69856459330143
<class 'str'>
SYMBOL :  EMMBI
HIT!
Growth :  115.56521739130434
<class 'str'>
SYMBOL :  IGL
HIT!
Growth :  860.6547619047619
----------------ALERT--------------
recDate :  2016-11-18 00:00:00
<class 'str'>
SYMBOL :  PETRONET
HIT!
Growth :  159.74063400576367
<class 'str'>
SYMBOL :  MONTECARLO
HIT!
Growth :  42.07459207459

<class 'str'>
SYMBOL :  CHOLAFIN
HIT!
Growth :  124.29446793820661
<class 'str'>
SYMBOL :  RBLBANK
HIT!
Growth :  91.5623475857584
<class 'str'>
SYMBOL :  EIDPARRY
HIT!
Growth :  44.92063492063491
<class 'str'>
SYMBOL :  KESORAMIND
DIDN'T HIT!
<class 'str'>
SYMBOL :  NBCC
HIT!
Growth :  83.91937432441023
<class 'str'>
SYMBOL :  RBLBANK
HIT!
Growth :  91.5623475857584
Broker :  AUM Capital HitRatio :  77 Avg. Growth :  70 Total Recos. :  9
brokerName :  Religare
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  49.93340375694668
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  49.93340375694668
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth :  205.4157103723995
----------------ALERT--------------
recDate :  2016-12-30 00:00:00
<class 'str'>
SYMBOL :  BHARATFIN
HIT!
Growth :  75.33919276388771
<class 'str'>
SYMBOL :  GLENMARK
DIDN'T HIT!
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BHARTIARTL
HIT!
Growth :  63.392991427282325
<class 'str'>
SYMBOL :  DIVISLAB
DIDN'T 

SYMBOL :  KOTAKBANK
HIT!
Growth :  30.568282842098064
<class 'str'>
SYMBOL :  TATAMOTORS
HIT!
Growth :  54.88029465930019
<class 'str'>
SYMBOL :  ZEEL
HIT!
Growth :  32.45995423340962
<class 'str'>
SYMBOL :  NIITTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  HEROMOTOCO
HIT!
Growth :  29.284766824909237
<class 'str'>
SYMBOL :  EMAMILTD
HIT!
Growth :  19.230585424133814
<class 'str'>
SYMBOL :  BHARATFIN
HIT!
Growth :  52.106045589692776
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  GODREJCP
HIT!
Growth :  170.68914081145587
<class 'str'>
SYMBOL :  SADBHAV
DIDN'T HIT!
<class 'str'>
SYMBOL :  MARICO
HIT!
Growth :  25.280681378242356
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  36.66451751558134
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  39.73181818181818
<class 'str'>
SYMBOL :  TCS
HIT!
Growth :  11.228675873273755
<class 'str'>
SYMBOL :  INFY
DIDN'T HIT!
<class 'str'>
SYMBOL :  CEATLTD
HIT!
Growth :  29.994222479000943
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth

SYMBOL :  KSL
HIT!
Growth :  130.43712898003238
<class 'str'>
SYMBOL :  HDFC
HIT!
Growth :  30.179344465058744
<class 'str'>
SYMBOL :  ITC
HIT!
Growth :  36.753843082451596
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  CHENNPETRO
HIT!
Growth :  107.51104565537555
<class 'str'>
SYMBOL :  ITC
HIT!
Growth :  54.48790508531422
<class 'str'>
SYMBOL :  HDFCBANK
DIDN'T HIT!
Broker :  Enochventures.com HitRatio :  52 Avg. Growth :  34 Total Recos. :  17
brokerName :  Edelweiss
<class 'str'>
SYMBOL :  KKCL
DIDN'T HIT!
<class 'str'>
SYMBOL :  TORNTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  JAGRAN
HIT!
Growth :  18.24404761904762
<class 'str'>
SYMBOL :  DEN
HIT!
Growth :  53.81944444444444
<class 'str'>
SYMBOL :  COALINDIA
DIDN'T HIT!
<class 'str'>
SYMBOL :  SIEMENS
HIT!
Growth :  33.11191992720656
<class 'str'>
SYMBOL :  SIMPLEXINF
HIT!
Growth :  99.24242424242425
<class 'str'>
SYMBOL :  EICHERMOT
HIT!
Growth :  48.01346725520222
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth : 

SYMBOL :  INFY
DIDN'T HIT!
<class 'str'>
SYMBOL :  LICHSGFIN
HIT!
Growth :  48.41835280706754
<class 'str'>
SYMBOL :  TORNTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  ASHOKA
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  23.588517676325967
<class 'str'>
SYMBOL :  LICHSGFIN
HIT!
Growth :  61.07216494845362
<class 'str'>
SYMBOL :  MINDACORP
HIT!
Growth :  23.25892857142858
<class 'str'>
SYMBOL :  TVSMOTOR
HIT!
Growth :  87.90657439446366
<class 'str'>
SYMBOL :  COALINDIA
DIDN'T HIT!
<class 'str'>
SYMBOL :  DEN
DIDN'T HIT!
<class 'str'>
SYMBOL :  BERGEPAINT
HIT!
Growth :  57.68531708994449
<class 'str'>
SYMBOL :  SAIL
HIT!
Growth :  60.59523809523811
<class 'str'>
SYMBOL :  JKCEMENT
HIT!
Growth :  91.69795221843002
<class 'str'>
SYMBOL :  DEEPAKFERT
HIT!
Growth :  88.14569536423842
<class 'str'>
SYMBOL :  RCOM
DIDN'T HIT!
<class 'str'>
SYMBOL :  TECHNOE
DIDN'T HIT!
<class 'str'>
SYMBOL :  SIMPLEXINF
HIT!
Growth :  56.831501831501825
<class 'str'>
SYMBOL :  SUPRAJIT
HIT!
Growth 

...

SYMBOL :  GSKCONS
DIDN'T HIT!
<class 'str'>
SYMBOL :  AMBUJACEM
HIT!
Growth :  19.486967314853132
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  SHREECEM
HIT!
Growth :  23.770491803278688
<class 'str'>
SYMBOL :  M&MFIN
HIT!
Growth :  23.561643835616437
<class 'str'>
SYMBOL :  JUBILANT
HIT!
Growth :  31.502311248073966
<class 'str'>
SYMBOL :  MARUTI
HIT!
Growth :  39.21245733788396
<class 'str'>
SYMBOL :  IOC
HIT!
Growth :  45.73717948717948
<class 'str'>
SYMBOL :  SRTRANSFIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  CUMMINSIND
HIT!
Growth :  25.123239436619713
<class 'str'>
SYMBOL :  JSWSTEEL
HIT!
Growth :  973.59413202934
----------------ALERT--------------
recDate :  2016-10-28 00:00:00
<class 'str'>
SYMBOL :  TECHM
DIDN'T HIT!
<class 'str'>
SYMBOL :  ABB
HIT!
Growth :  50.700846660395115
<class 'str'>
SYMBOL :  HEROMOTOCO
HIT!
Growth :  22.140271493212666
<class 'str'>
SYMBOL :  PERSISTENT
DIDN'T HIT!
<class 'str'>
SYMBOL :  RALLIS
HIT!
Growth :  10.110216193302245
<cla

SYMBOL :  HEIDELBERG
DIDN'T HIT!
<class 'str'>
SYMBOL :  TATACOFFEE
HIT!
Growth :  30.720000000000002
<class 'str'>
SYMBOL :  DCMSHRIRAM
HIT!
Growth :  76.62423178226514
<class 'str'>
SYMBOL :  CYIENT
DIDN'T HIT!
<class 'str'>
SYMBOL :  RECLTD
HIT!
Growth :  129.51197053407
<class 'str'>
SYMBOL :  LUPIN
HIT!
Growth :  21.667832167832163
<class 'str'>
SYMBOL :  KRBL
HIT!
Growth :  48.123953098827464
<class 'str'>
SYMBOL :  PRAJIND
HIT!
Growth :  14.545454545454543
<class 'str'>
SYMBOL :  TORNTPOWER
HIT!
Growth :  29.945054945054945
<class 'str'>
SYMBOL :  BIOCON
HIT!
Growth :  472.84345047923324
----------------ALERT--------------
recDate :  2016-05-11 00:00:00
<class 'str'>
SYMBOL :  NOCIL
HIT!
Growth :  107.14285714285714
<class 'str'>
SYMBOL :  WELSPUNIND
HIT!
Growth :  17.936364548830678
Broker :  Dynamic Levels HitRatio :  84 Avg. Growth :  65 Total Recos. :  53
brokerName :  Choice Equity Broking
<class 'str'>
SYMBOL :  HCLTECH
HIT!
Growth :  16.718555417185556
<class 'str'>
SYMBO

SYMBOL :  TVSMOTOR
HIT!
Growth :  102.18842154275961
<class 'str'>
SYMBOL :  CEATLTD
HIT!
Growth :  85.29326923076923
<class 'str'>
SYMBOL :  AJANTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  EMAMILTD
DIDN'T HIT!
<class 'str'>
SYMBOL :  AJANTPHARM
HIT!
Growth :  30.395913154533847
<class 'str'>
SYMBOL :  JKIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  APLLTD
HIT!
Growth :  25.164533820840948
<class 'str'>
SYMBOL :  KEC
HIT!
Growth :  92.01438848920861
<class 'str'>
SYMBOL :  TALWALKARS
DIDN'T HIT!
<class 'str'>
SYMBOL :  RKFORGE
DIDN'T HIT!
<class 'str'>
SYMBOL :  RSWM
HIT!
Growth :  28.643410852713185
<class 'str'>
SYMBOL :  YESBANK
HIT!
Growth :  737.0881567230632
----------------ALERT--------------
recDate :  2016-11-24 00:00:00
<class 'str'>
SYMBOL :  TVSMOTOR
HIT!
Growth :  102.18842154275961
<class 'str'>
SYMBOL :  AJANTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  AJANTPHARM
HIT!
Growth :  30.395913154533847
<class 'str'>
SYMBOL :  JKIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  KEC
HIT!
Growth :  9

SYMBOL :  UPL
HIT!
Growth :  58.69525877669201
<class 'str'>
SYMBOL :  MARUTI
HIT!
Growth :  71.1687271062271
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  42.168473030087476
<class 'str'>
SYMBOL :  INDUSINDBK
HIT!
Growth :  38.728888888888896
<class 'str'>
SYMBOL :  NAVKARCORP
DIDN'T HIT!
<class 'str'>
SYMBOL :  INDIACEM
HIT!
Growth :  112.39065654138227
<class 'str'>
SYMBOL :  NBCC
HIT!
Growth :  54.020234291799774
<class 'str'>
SYMBOL :  PERSISTENT
DIDN'T HIT!
<class 'str'>
SYMBOL :  ITC
HIT!
Growth :  53.88148828663298
<class 'str'>
SYMBOL :  INDIACEM
HIT!
Growth :  111.16528056591149
<class 'str'>
SYMBOL :  HCLTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  37.04741379310344
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  28.56147155974422
<class 'str'>
SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BAJAJ-AUTO
HIT!
Growth :  15.160344767421584
<class 'str'>
SYMBOL :  GODREJCP
HIT!
Growth :  156.94937541091386
<class 'str'>
SYMBOL :  NESTLEIND
HIT!
Gr

SYMBOL :  ESCORTS
HIT!
Growth :  117.45664739884393
<class 'str'>
SYMBOL :  HCC
HIT!
Growth :  68.92857142857142
<class 'str'>
SYMBOL :  MOTHERSUMI
HIT!
Growth :  130.85256712199717
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  26.45089443042614
<class 'str'>
SYMBOL :  KNRCON
HIT!
Growth :  624.4594594594595
----------------ALERT--------------
recDate :  2016-06-01 00:00:00
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  30.334869431643614
<class 'str'>
SYMBOL :  JYOTHYLAB
HIT!
Growth :  25.1
<class 'str'>
SYMBOL :  JBMA
HIT!
Growth :  117.55244755244757
<class 'str'>
SYMBOL :  ESCORTS
HIT!
Growth :  117.45664739884393
<class 'str'>
SYMBOL :  HCC
HIT!
Growth :  68.92857142857142
<class 'str'>
SYMBOL :  MOTHERSUMI
HIT!
Growth :  130.85256712199717
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  26.45089443042614
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  30.334869431643614
Broker :  Karvy 

In [24]:
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos
0,KRChoksey,69,50,117
1,AUM Capital,77,70,9
2,Religare,79,36,222
3,Enochventures.com,52,34,17
4,Edelweiss,72,35,278
5,Motilal Oswal,69,37,299
6,Kotak Securities,82,43,75
7,Angel Broking,54,36,66
8,Geojit Financial Services,83,42,118
9,Centrum,72,40,127


In [26]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2016', if_exists='replace')